 model choice (why doble binary - case sensitivity only defined for binary classification and we care abot that more than acc, introdce sensitivity formlas, model strct - modified resnet why? print model)
- training and validation (rn train py and show loss curve, talk abot adjusting NLLLOSS)

# Choice of Overall Classifier Architecture:
From researching the topic, our group was initially unable to form a definitive conclusion since most answers seemed to (vaguely) point that the best approach would depend on the nature of the problem, the amount of data available, and computation power.

Hence, our group decided to build out both architectures with a relatively simple model as an experiment (ScrappyNet in train.py - see below). While the multiclass classifier achieved a higher overall accuracy **(79% vs 64% cascaded)**, we realised that it would be far easier to tweak two binary classifiers separately.

Our rationale for this is that our overall goal is build a model with high sensitivity (rather than accuracy) to pick out infected cases from the data and ensure that these patients can seek medical treatment. Additionally, as Covid-19 is highly contagious, there is an impetus to maximise the sensitivity of these from the infected cases. Hence, the cascaded binary classifier structure naturally aligns with this by allowing us to finetune for sensitivity at each stage.

Below is a brief summary of what we believe to be the tradeoffs of each model, although we have selected the cascaded architecture.

## Comparison Summary 
#### Multiclass Pros
- Simple to evaluate and tweak since we only have to deal with a single model
- Seemed to be able to obtain higher overall accuracy than the alternative on this particular dataset

#### Multiclass Cons
- Difficult to balance the relationship between the three classes, which is crucial for sensitivity tuning

#### Cascaded Binary Pros
- Easy to balance weights of different classifications in a step-wise process to fine tune sensitivity
- Converges faster on a per-model basis
- More accurate in theory due to the potential of modeling stronger relationships between pair-wise classes

#### Cascaded Binary Cons
- Irritating to tweak and evaluate, in addition to needing extra steps to build the dataloaders for each one
- Overall time to train is longer due to the presence of two models


Logically speaking, this may be why doctors in practice do not use x-ray scans to classify the source of lung damage, but only to confirm the presence of it. Hence, this exercise of actually determining what caused the lung damage is a fallacy in itself, and the best way to account for this is to use a separate classifier.

### Dataloader prep for below section

In [2]:
from dataset import BinaryClassDataset, TrinaryClassDataset
from torch.utils.data import DataLoader, ConcatDataset

trainingBatchSize = 4
img_size = (150, 150)
class_dict = {0: 'normal', 1: 'infected'}
groups = ['train']
dataset_numbers = {'train_normal': 0,
                   'train_infected': 2530,
                   }

dataset_paths = {'train_normal': './dataset/train/normal/',
                 'train_infected': './dataset/train/infected/non-covid',
                 }

trainset1 = BinaryClassDataset('train', img_size, class_dict, groups, dataset_numbers, dataset_paths)

dataset_numbers = {'train_normal': 1341,
                   'train_infected': 1345,
                   }

dataset_paths = {'train_normal': './dataset/train/normal/',
                 'train_infected': './dataset/train/infected/covid',
                 }

trainset2 = BinaryClassDataset('train', img_size, class_dict, groups, dataset_numbers, dataset_paths)

# load dataset
trainsets = ConcatDataset([trainset1, trainset2])
trainloader = DataLoader(trainsets, batch_size=trainingBatchSize, shuffle=True)

val_groups = ['val']
val_numbers = {'val_normal': 8,
               'val_infected': 8,
               }

valset_paths = {'val_normal': './dataset/test/normal',
                'val_infected': './dataset/test/infected/covid',
                }

valset1 = BinaryClassDataset('val', img_size, class_dict, val_groups, val_numbers, valset_paths)

val_numbers = {'val_normal': 0,
               'val_infected': 8,
               }

valset_paths = {'val_normal': './dataset/val/normal',
                'val_infected': './dataset/val/infected/non-covid',
                }

valset2 = BinaryClassDataset('val', img_size, class_dict, val_groups, val_numbers, valset_paths)

# load dataset
valsets = ConcatDataset([valset1, valset2])
validationloader = DataLoader(valsets, batch_size=trainingBatchSize, shuffle=True)

# Choice of CNN Architecture
Rather than conceive a completely new architecture from our limited knowledge, our group adopted an approach where we trained and evaluated a few of the available predefined models in torchvision, then selected the best contender to build from scratch so that we could tweak it.

Given the constraints of 1) a relatively small training dataset and 2) limited computation power, we decided to keep the number of parameters on the lower side to follow the golden rule of machine learning. Hence, from the illustration below plotting top-1 accuracy against the number of operations, we selected the ResNet, DenseNet, MobileNet, and Inception architectures. Specifically, we chose ResNet-18, DenseNet-121, MobileNetV2 and Inception-v3.

<img src="https://miro.medium.com/max/4000/1*n16lj3lSkz2miMc_5cvkrA.jpeg" style="height:300px">

After testing the models, we found that ResNet-18 was the clear winner for our base model, achieving ~90% accuracy on the validation set with the lowest loss on the training set. 
 
- DenseNet performed well, but required significantly more time to train than ResNet - more than 3x the time!
- Inception showed subpar performance, and like DenseNet required more time to train than ResNet
- MobileNet unfortunately did not seem to have good validation accuracy despite having a decent loss. It also surprisingly took longer to train than ResNet.

In [3]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms

from train import train, validate

resnet = models.resnet18(pretrained=False)
num_ftrs = resnet.fc.in_features
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet.fc = nn.Linear(num_ftrs, 2)
# print(resnet)

densenet = models.densenet121(pretrained=False)
num_ftrs = densenet.classifier.in_features
densenet.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
densenet.classifier = nn.Linear(num_ftrs, 2)
# print(densenet)

inception = models.inception_v3(pretrained=False, aux_logits=False) #disable auxiliary channel to accept 150x150 images
inception.Conv2d_1a_3x3.conv=nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
num_ftrs = inception.fc.in_features
inception.fc = nn.Linear(num_ftrs,2)
# print(inception)

mobilenet = models.mobilenet_v2(pretrained=False)
mobilenet.features[0][0]=nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
num_ftrs = mobilenet.classifier[1].in_features
mobilenet.classifier[1] = nn.Linear(num_ftrs, 2)
# print(mobilenet)

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\inception.py:75: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of inception_v3 will be changed in future releases of '


In [4]:
weight=None
start= time.time()
savePath='models/testingResnet'
lowest_loss=9999
for epoch in range(1, 11):
    train(resnet, trainloader, weight, epoch)
    lowest_loss = validate(resnet, validationloader, weight, epoch, lowest_loss, savePath)
    
time.strftime("%Mmin %Ss", time.gmtime(time.time()-start))

Train Epoch: 1
Train Epoch: 1 [0/5216 (0%)]	Loss: 3.126391
Train Epoch: 1 [400/5216 (8%)]	Loss: 1.688136
Train Epoch: 1 [800/5216 (15%)]	Loss: 1.464071
Train Epoch: 1 [1200/5216 (23%)]	Loss: 2.334665
Train Epoch: 1 [1600/5216 (31%)]	Loss: 1.586831
Train Epoch: 1 [2000/5216 (38%)]	Loss: 4.504718
Train Epoch: 1 [2400/5216 (46%)]	Loss: 1.890346
Train Epoch: 1 [2800/5216 (54%)]	Loss: 3.147740
Train Epoch: 1 [3200/5216 (61%)]	Loss: 2.980558
Train Epoch: 1 [3600/5216 (69%)]	Loss: 1.539777
Train Epoch: 1 [4000/5216 (77%)]	Loss: 3.115664
Train Epoch: 1 [4400/5216 (84%)]	Loss: 0.471093
Train Epoch: 1 [4800/5216 (92%)]	Loss: 1.300955
Train Epoch: 1 [5200/5216 (100%)]	Loss: 4.177063
Validation set: Average loss: 0.0204, Accuracy: 13/24 (54%)

Found New Minima at epoch 1 loss: 0.020439488020005808

Train Epoch: 2
Train Epoch: 2 [0/5216 (0%)]	Loss: 2.044962
Train Epoch: 2 [400/5216 (8%)]	Loss: 1.106435
Train Epoch: 2 [800/5216 (15%)]	Loss: 2.632635
Train Epoch: 2 [1200/5216 (23%)]	Loss: 0.733023
Tr

'06min 31s'

In [5]:
start= time.time()
savePath='models/testingDensenet'
lowest_loss=9999
for epoch in range(1, 11):
    train(densenet, trainloader, weight, epoch)
    lowest_loss = validate(densenet, validationloader, weight, epoch, lowest_loss, savePath)

time.strftime("%Mmin %Ss", time.gmtime(time.time()-start))

Train Epoch: 1
Train Epoch: 1 [0/5216 (0%)]	Loss: 2.205317
Train Epoch: 1 [400/5216 (8%)]	Loss: 2.154171
Train Epoch: 1 [800/5216 (15%)]	Loss: 2.856686
Train Epoch: 1 [1200/5216 (23%)]	Loss: 4.135695
Train Epoch: 1 [1600/5216 (31%)]	Loss: 1.733558
Train Epoch: 1 [2000/5216 (38%)]	Loss: 2.008223
Train Epoch: 1 [2400/5216 (46%)]	Loss: 1.516551
Train Epoch: 1 [2800/5216 (54%)]	Loss: 1.584389
Train Epoch: 1 [3200/5216 (61%)]	Loss: 2.246727
Train Epoch: 1 [3600/5216 (69%)]	Loss: 1.273621
Train Epoch: 1 [4000/5216 (77%)]	Loss: 1.375802
Train Epoch: 1 [4400/5216 (84%)]	Loss: 1.265940
Train Epoch: 1 [4800/5216 (92%)]	Loss: 3.649611
Train Epoch: 1 [5200/5216 (100%)]	Loss: 1.009843
Validation set: Average loss: 0.0318, Accuracy: 17/24 (71%)

Found New Minima at epoch 1 loss: 0.031779092092280005

Train Epoch: 2
Train Epoch: 2 [0/5216 (0%)]	Loss: 1.255838
Train Epoch: 2 [400/5216 (8%)]	Loss: 0.953703
Train Epoch: 2 [800/5216 (15%)]	Loss: 1.700947
Train Epoch: 2 [1200/5216 (23%)]	Loss: 0.986817
Tr

'21min 40s'

In [6]:
start= time.time()
savePath='models/testingInception'
lowest_loss=9999
for epoch in range(1, 11):
    train(inception, trainloader, weight, epoch)
    lowest_loss = validate(inception, validationloader, weight, epoch, lowest_loss, savePath)

time.strftime("%Mmin %Ss", time.gmtime(time.time()-start))

Train Epoch: 1
Train Epoch: 1 [0/5216 (0%)]	Loss: 2.371321
Train Epoch: 1 [400/5216 (8%)]	Loss: 2.902738
Train Epoch: 1 [800/5216 (15%)]	Loss: 2.360368
Train Epoch: 1 [1200/5216 (23%)]	Loss: 2.703707
Train Epoch: 1 [1600/5216 (31%)]	Loss: 1.491593
Train Epoch: 1 [2000/5216 (38%)]	Loss: 1.484870
Train Epoch: 1 [2400/5216 (46%)]	Loss: 1.384418
Train Epoch: 1 [2800/5216 (54%)]	Loss: 0.819847
Train Epoch: 1 [3200/5216 (61%)]	Loss: 1.256956
Train Epoch: 1 [3600/5216 (69%)]	Loss: 1.143578
Train Epoch: 1 [4000/5216 (77%)]	Loss: 1.736389
Train Epoch: 1 [4400/5216 (84%)]	Loss: 2.524489
Train Epoch: 1 [4800/5216 (92%)]	Loss: 4.880476
Train Epoch: 1 [5200/5216 (100%)]	Loss: 1.145750
Validation set: Average loss: 0.0319, Accuracy: 16/24 (67%)

Found New Minima at epoch 1 loss: 0.03185489300397238

Train Epoch: 2
Train Epoch: 2 [0/5216 (0%)]	Loss: 2.685824
Train Epoch: 2 [400/5216 (8%)]	Loss: 0.653553
Train Epoch: 2 [800/5216 (15%)]	Loss: 2.544520
Train Epoch: 2 [1200/5216 (23%)]	Loss: 0.349706
Tra

'16min 18s'

In [7]:
start= time.time()
savePath='models/testingMobilenet'
lowest_loss=9999
for epoch in range(1, 11):
    train(mobilenet, trainloader, weight, epoch)
    lowest_loss = validate(mobilenet, validationloader, weight, epoch, lowest_loss, savePath)
    
time.strftime("%Mmin %Ss", time.gmtime(time.time()-start))

Train Epoch: 1
Train Epoch: 1 [0/5216 (0%)]	Loss: 2.743382
Train Epoch: 1 [400/5216 (8%)]	Loss: 2.534216
Train Epoch: 1 [800/5216 (15%)]	Loss: 2.436188
Train Epoch: 1 [1200/5216 (23%)]	Loss: 1.987141
Train Epoch: 1 [1600/5216 (31%)]	Loss: 2.909914
Train Epoch: 1 [2000/5216 (38%)]	Loss: 2.030223
Train Epoch: 1 [2400/5216 (46%)]	Loss: 1.904934
Train Epoch: 1 [2800/5216 (54%)]	Loss: 1.794584
Train Epoch: 1 [3200/5216 (61%)]	Loss: 1.617892
Train Epoch: 1 [3600/5216 (69%)]	Loss: 1.832922
Train Epoch: 1 [4000/5216 (77%)]	Loss: 3.565182
Train Epoch: 1 [4400/5216 (84%)]	Loss: 1.456826
Train Epoch: 1 [4800/5216 (92%)]	Loss: 2.200696
Train Epoch: 1 [5200/5216 (100%)]	Loss: 1.371087
Validation set: Average loss: 0.0418, Accuracy: 14/24 (58%)

Found New Minima at epoch 1 loss: 0.04178734235751722

Train Epoch: 2
Train Epoch: 2 [0/5216 (0%)]	Loss: 1.884462
Train Epoch: 2 [400/5216 (8%)]	Loss: 0.941054
Train Epoch: 2 [800/5216 (15%)]	Loss: 1.375443
Train Epoch: 2 [1200/5216 (23%)]	Loss: 2.579613
Tra

'08min 34s'

# Final Modified ResNet:
The final model we used is a modified 10 layer version of ResNet-18. The rationale for this is that the input images to this model (150 x 150) are smaller than the typical images used in ResNet(224x224), and hence have less features that need to be learned. Additionally, the full ResNet-18 also showed overfitting on the second binary model, and decreasing the layers would allow the model to generalise better.

We experimented with adding dropout layers, but only saw decreases in overall performance.

In [8]:
from model import ResNet
model=ResNet()
print(model)

ResNet(
  (layers): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (5): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2)

# Choice of loss function and optimizer
Our choice of loss function is (crossentropy or nnl?)



In [9]:
# getting normalization value
train_data = DataLoader(trainsets, batch_size=(2530+1341+1345), shuffle=True)
data = next(iter(train_data))
data[0].mean(), data[0].std()


(tensor(0.4824), tensor(0.2363))